In [2]:
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.widgets import Button, RadioButtons
import matplotlib
from tkinter.colorchooser import askcolor
from mpl_toolkits.mplot3d import Axes3D

matplotlib.use('TkAgg')

# Create a blank canvas
canvas = np.ones((500, 500, 3))  # White canvas (1 = white for RGB)

# Layers
layer1 = np.ones((500, 500, 3))
layer2 = np.ones((500, 500, 3))
current_layer = layer1

# Create a Matplotlib figure
fig, ax = plt.subplots()
im = ax.imshow(canvas, vmin=0, vmax=1)
plt.axis("off")

# Initialize variables
drawing = False
brush_size = 5
brush_color = (0, 0, 0)  # Default black
shape_mode = "Circle"  # Default shape
zoom_scale = 1.0

# Function to draw shapes
def draw_shape(x, y, size, color, shape="Circle"):
    global current_layer
    x, y = int(x), int(y)
    for i in range(-size, size + 1):
        for j in range(-size, size + 1):
            if 0 <= x + i < current_layer.shape[1] and 0 <= y + j < current_layer.shape[0]:
                if shape == "Circle" and i**2 + j**2 <= size**2:
                    current_layer[y + j, x + i] = color
                elif shape == "Rectangle":
                    current_layer[y + j, x + i] = color
                elif shape == "Ellipse" and (i / size)**2 + (j / size / 2)**2 <= 1:
                    current_layer[y + j, x + i] = color

    update_canvas()

# Function to update the combined canvas
def update_canvas():
    global canvas
    canvas[:] = np.minimum(layer1, layer2)
    im.set_data(canvas)
    fig.canvas.draw_idle()

# Function to generate 3D view
def generate_3d(event):
    global canvas
    fig_3d = plt.figure(figsize=(10, 7))
    ax_3d = fig_3d.add_subplot(111, projection='3d')

    # Create height map from canvas intensity
    height_map = 1 - np.mean(canvas, axis=2)  # Average over RGB channels
    x = np.arange(canvas.shape[1])
    y = np.arange(canvas.shape[0])
    x, y = np.meshgrid(x, y)

    # Apply surface shading and texture
    ax_3d.plot_surface(x, y, height_map, facecolors=canvas, rstride=1, cstride=1, antialiased=False, shade=False)

    ax_3d.set_title("3D View of the Drawing")
    ax_3d.set_xlabel("X-axis")
    ax_3d.set_ylabel("Y-axis")
    ax_3d.set_zlabel("Height")
    plt.show()

# Mouse press handler
def on_press(event):
    global drawing
    if event.xdata is not None and event.ydata is not None:
        drawing = True
        draw_shape(event.xdata, event.ydata, brush_size, brush_color, shape_mode)

# Mouse release handler
def on_release(event):
    global drawing
    drawing = False

# Mouse motion handler
def on_motion(event):
    if drawing and event.xdata is not None and event.ydata is not None:
        draw_shape(event.xdata, event.ydata, brush_size, brush_color, shape_mode)

# Keyboard handler for brush size adjustment
def on_key(event):
    global brush_size
    if event.key == "up":
        brush_size += 1
    elif event.key == "down":
        brush_size = max(1, brush_size - 1)
    print(f"Brush size: {brush_size}")

# Function to pick a brush color
def pick_color(event):
    global brush_color
    color = askcolor()[0]
    if color:
        brush_color = tuple(c / 255 for c in color)
    print(f"Brush color: {brush_color}")

# Zooming
def on_scroll(event):
    global zoom_scale
    if event.button == "up":
        zoom_scale *= 1.1
    elif event.button == "down":
        zoom_scale /= 1.1
    ax.set_xlim(ax.get_xlim()[0] * zoom_scale, ax.get_xlim()[1] * zoom_scale)
    ax.set_ylim(ax.get_ylim()[0] * zoom_scale, ax.get_ylim()[1] * zoom_scale)
    fig.canvas.draw_idle()

# Layer toggle
def toggle_layer(event):
    global current_layer
    current_layer = layer1 if current_layer is layer2 else layer2
    print("Toggled layer")
    update_canvas()

# Shape selection
def set_shape(label):
    global shape_mode
    shape_mode = label
    print(f"Selected shape: {shape_mode}")

# Add UI elements
color_ax = plt.axes([0.8, 0.2, 0.1, 0.075])
color_button = Button(color_ax, "Pick Color")
color_button.on_clicked(pick_color)

toggle_ax = plt.axes([0.8, 0.1, 0.1, 0.075])
toggle_button = Button(toggle_ax, "Toggle Layer")
toggle_button.on_clicked(toggle_layer)

shape_ax = plt.axes([0.8, 0.3, 0.15, 0.15])
shape_radio = RadioButtons(shape_ax, ["Circle", "Rectangle", "Ellipse"])
shape_radio.on_clicked(set_shape)

view_3d_ax = plt.axes([0.8, 0.05, 0.1, 0.075])
view_3d_button = Button(view_3d_ax, "3D View")
view_3d_button.on_clicked(generate_3d)

# Connect events
fig.canvas.mpl_connect("button_press_event", on_press)
fig.canvas.mpl_connect("button_release_event", on_release)
fig.canvas.mpl_connect("motion_notify_event", on_motion)
fig.canvas.mpl_connect("key_press_event", on_key)
fig.canvas.mpl_connect("scroll_event", on_scroll)

# Show the canvas
plt.show()

2025-02-25 14:04:54.675 python[68825:4410217] +[IMKClient subclass]: chose IMKClient_Modern
2025-02-25 14:04:54.675 python[68825:4410217] +[IMKInputSession subclass]: chose IMKInputSession_Modern
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.12/site-packages/matplotlib/cbook.py", line 298, in process
    func(*args, **kwargs)
  File "/opt/anaconda3/lib/python3.12/site-packages/matplotlib/widgets.py", line 231, in _click
    event.canvas.grab_mouse(self.ax)
  File "/opt/anaconda3/lib/python3.12/site-packages/matplotlib/backend_bases.py", line 1837, in grab_mouse
    raise RuntimeError("Another Axes already grabs mouse input")
RuntimeError: Another Axes already grabs mouse input


Toggled layer
